# EasyUQ on weather example

    1. Ensemble smoothing 
    2. Single gaussian applied to WeatherBench data (here: CNN)
    3. Smooth EasyUQ applied to WeatherBench data (here: CNN)

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import isodisreg 
from isodisreg import idr
from scipy import stats
import properscoring as ps
from helper_functions import ensemble_smoothing, llscore_ens_smoothing
#, crps_ens_smoothing

### 1. Ensemble smoothing: get ensemble data from isodisreg package

In [2]:
# Get data
rain = isodisreg.load_rain()
varNames = rain.columns[3:55]

ensemble = rain[varNames].to_numpy()
y = rain['obs'].to_numpy()

ensemble_train = ensemble[:2887, ]
ensemble_test = ensemble[2887:]
y_train = y[:2887]
y_test = y[2887:]

# CRPS of ensemble
crps_ens = np.mean(ps.crps_ensemble(y_test, ensemble_test))
print('CRPS of ensemble: %f' %crps_ens)

CRPS of ensemble: 0.755056


Run ensemble smoothing and output log score and CRPS

In [4]:
from crpsmixture import smooth_crps_ens

In [4]:
ll_train, h, df = ensemble_smoothing(ensemble_train, y_train)
ll = llscore_ens_smoothing(ensemble_test, y_test, h, df)
crpss = crps_ens_smoothing(ensemble_test, y_test, h, df)
print('Fitted df = %d' %df , 'and h = %f' %h)
print('CRPS of smooth ensemble: %f' %crpss)
print('log score: %f' %ll)

Fitted df = 2 and h = 0.231860
CRPS of smooth ensemble: 0.761099
log score: 1.119998


### 2. Single gaussian

Get CNN forecast from WeatherBench to apply single gaussian and smooth EasyUQ

In [ ]:
# load via xarray:
cnn_forecast_train = xr.open_dataset()
cnn_forecast_test = xr.open_dataset()
y_train = xr.open_dataset()
y_test = xr.open_dataset()

# select one grid point:
lat = 13
lon = 52
cnn_fct_grid_train = cnn_forecast_train.sel(lat = lat, lon = lon)['t'].values
cnn_fct_grid_test = cnn_forecast_test.sel(lat = lat, lon = lon)['t'].values
y_grid_train = y_train.sel(lat = lat, lon = lon)['t'].values
y_grid_test = y_test.sel(lat = lat, lon = lon)['t'].values

Run single gaussian method

In [ ]:
sigma = single_gaussian_optim(y_grid_train, cnn_fct_grid_train)
crpss = np.mean(ps.crps_gaussian(y_grid_test, mu=cnn_fct_grid_test, sig=sigma)) 
ll = log_norm(y_grid_test, cnn_fct_grid_test, sigma)
print('Fitted sigma = %f' %sigma)
print('CRPS of single gaussian: %f' %crpss)
print('log score: %f' %ll)

### 3. Smooth EasyUQ via One-fit approach

Get CNN forecast from WeatherBench to apply single gaussian and smooth EasyUQ

In [ ]:
# load via xarray:
cnn_forecast_train = xr.open_dataset()
cnn_forecast_test = xr.open_dataset()
y_train = xr.open_dataset()
y_test = xr.open_dataset()

# select one grid point:
lat = 13
lon = 52
cnn_fct_grid_train = cnn_forecast_train.sel(lat = lat, lon = lon)['t'].values
cnn_fct_grid_test = cnn_forecast_test.sel(lat = lat, lon = lon)['t'].values
y_grid_train = y_train.sel(lat = lat, lon = lon)['t'].values
y_grid_test = y_test.sel(lat = lat, lon = lon)['t'].values

In [ ]:
# Fit EasyUQ
fitted_idr = idr(y_grid_train, pd.DataFrame({"fore": cnn_fct_grid_train}, columns=["fore"]))
preds_train = fitted_idr.predict(pd.DataFrame({"fore": cnn_fct_grid_train}, columns=["fore"]))
preds_test = fitted_idr.predict(pd.DataFrame({"fore": cnn_fct_grid_test}, columns=["fore"]))

In [ ]:
ll_train, h, df = optimize_paras_onefit(preds_train, y_grid_train)
crpss = smooth_crps(preds_test, y_grid_test, h, df) 
ll = llscore(preds_test, y_grid_test, h, df)
print('Fitted df = %d' %df , 'and h = %f' %h)
print('CRPS of smooth EasyUQ: %f' %crpss)
print('log score: %f' %ll)